**1. Mount google drive**


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

**2. Download tensorflow model garden**

In [ ]:
#cd into the TensorFlow directory in your Google Drive
%cd '/content/gdrive/My Drive/TensorFlow'

#and clone the TensorFlow Model Garden repository
!git clone https://github.com/tensorflow/models.git

#using a older version of repo
%cd '/content/gdrive/MyDrive/TensorFlow/models'
!git checkout -f e04dafd04d69053d3733bb91d47d0d95bc2c8199


**3. Install some required libraries and tools**

In [ ]:
!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim lvis

**4. Compile the protobuf libraries**

In [ ]:
#cd into 'TensorFlow/models/research'
%cd '/content/gdrive/My Drive/TensorFlow/models/research/'
!protoc object_detection/protos/*.proto --python_out=.

**5. Set the environment**

In [ ]:
import os
import sys

os.environ['PYTHONPATH']+=":/content/gdrive/My Drive/TensorFlow/models"

sys.path.append("/content/gdrive/My Drive/TensorFlow/models/research")

**6. Build and install setup.py**

In [ ]:
%cd '/content/gdrive/My Drive/TensorFlow/models/research/'
!python setup.py build
!python setup.py install

*and test the installation*

In [ ]:
#cd into 'TensorFlow/models/research/object_detection/builders/'
%cd '/content/gdrive/My Drive/TensorFlow/models/research/object_detection/builders/'
!python model_builder_tf2_test.py
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
print('Done')


**7. Start TensorBoard**

In [ ]:
#cd into training_demo
%cd '/content/gdrive/My Drive/TensorFlow/workspace/training_demo'

#start the Tensorboard

%tensorboard --logdir=models/my_ssd_resnet50_v1_fpn_640x640
%reload_ext tensorboard

# %load_ext tensorboard
# %tensorboard --logdir=models/[name_of_pre-trained-model_you_downloaded]

In [ ]:
#cd into training_demo
%cd '/content/gdrive/My Drive/TensorFlow/workspace/training_demo'

#start the Tensorboard
%load_ext tensorboard
%tensorboard --logdir=models/my_ssd_mobilenet_v1_fpn_640x640

# %load_ext tensorboard
# %tensorboard --logdir=models/[name_of_pre-trained-model_you_downloaded]

**8. Train the model**

In [ ]:
!nvidia-smi

In [ ]:
!cat /proc/cpuinfo

In [ ]:
!pip install opencv-python-headless==4.1.2.30

In [ ]:
#cd into training_demo
%cd '/content/gdrive/My Drive/TensorFlow/workspace/training_demo'

*Train ssd_mobilenet_v2_2* (Just an exercise)

In [ ]:
!python model_main_tf2.py --model_dir=models/my_ssd_mobilenet_v2_2 --pipeline_config_path=models/my_ssd_mobilenet_v2_2/pipeline.config
# !python model_main_tf2.py --model_dir=models/[name_of_pre-trained-model_you_downloaded] --pipeline_config_path=models/[name_of_pre-trained-model_you_downloaded]/pipeline.config

*Train ssd_resnet50_v1_fpn_640x640*

In [ ]:
!python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn_640x640 --pipeline_config_path=models/my_ssd_resnet50_v1_fpn_640x640/pipeline.config
# !python model_main_tf2.py --model_dir=models/[name_of_pre-trained-model_you_downloaded] --pipeline_config_path=models/[name_of_pre-trained-model_you_downloaded]/pipeline.config

*Train ssd_mobilenet_v1_fpn_640x640*

In [ ]:
!python model_main_tf2.py --model_dir=models/my_ssd_mobilenet_v1_fpn_640x640 --pipeline_config_path=models/my_ssd_mobilenet_v1_fpn_640x640/pipeline.config
# !python model_main_tf2.py --model_dir=models/[name_of_pre-trained-model_you_downloaded] --pipeline_config_path=models/[name_of_pre-trained-model_you_downloaded]/pipeline.config

**9. Export the trained model**

*Export ssd_resnet50_v1_fpn_640x640*

In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/my_ssd_resnet50_v1_fpn_640x640/pipeline.config --trained_checkpoint_dir ./models/my_ssd_resnet50_v1_fpn_640x640/ --output_directory ./exported-models/my_ssd_resnet50_v1_fpn_640x640
# !python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/[name_of_pre-trained-model you downloaded]/pipeline.config --trained_checkpoint_dir ./models/[name_of_pre-trained-model_you_downloaded]/ --output_directory ./exported-models/my_model

*Export ssd_mobilenet_v1_fpn_640x640*

In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/my_ssd_mobilenet_v1_fpn_640x640/pipeline.config --trained_checkpoint_dir ./models/my_ssd_mobilenet_v1_fpn_640x640/ --output_directory ./exported-models/my_ssd_mobilenet_v1_fpn_640x640
# !python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/[name_of_pre-trained-model you downloaded]/pipeline.config --trained_checkpoint_dir ./models/[name_of_pre-trained-model_you_downloaded]/ --output_directory ./exported-models/my_model

**10. Testing the model**

loading saved_model

In [ ]:
#Loading the saved_model(change the path according to your directory names)
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
PATH_TO_SAVED_MODEL="/content/gdrive/My Drive/TensorFlow/workspace/training_demo/exported-models/my_ssd_mobilenet_v1_fpn_640x640/saved_model"
print('Loading model...', end='')
# Load saved model and build the detection function
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)
print('Done!')

loading model_map

In [ ]:
#Loading the label_map
category_index=label_map_util.create_category_index_from_labelmap("/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt",use_display_name=True)
#category_index=label_map_util.create_category_index_from_labelmap([path_to_label_map],use_display_name=True)

loading images

In [ ]:
#Loading the image
img=['/content/gdrive/My Drive/test_image/735.tif']
print(img)
#list containing paths of all the images

**11. Runing the inference**

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
def load_image_into_numpy_array(path):
    return np.array(Image.open(path))

for image_path in img:
    print('Running inference for {}... '.format(image_path), end='')
    image_np=load_image_into_numpy_array(image_path)
    input_tensor=tf.convert_to_tensor(image_np)
    input_tensor=input_tensor[tf.newaxis, ...]
    detections=detect_fn(input_tensor)

num_detections=int(detections.pop('num_detections'))
detections={key:value[0,:num_detections].numpy() for key,value in detections.items()}
detections['num_detections']=num_detections

detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_np_with_detections=image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=100,     
          min_score_thresh=.5,      
          agnostic_mode=False)

%matplotlib inline
plt.figure()
plt.imshow(image_np_with_detections)
print('Done')
plt.show()

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
def load_image_into_numpy_array(path):
    return np.array(Image.open(path))

for image_path in img:
    print('Running inference for {}... '.format(image_path), end='')
    image_np=load_image_into_numpy_array(image_path)
    input_tensor=tf.convert_to_tensor(image_np)
    input_tensor=input_tensor[tf.newaxis, ...]
    detections=detect_fn(input_tensor)
  
detections